In [1]:
import argparse
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import save_image
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt

from custom_dataset import CustomImageDataset

In [29]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.ConvTranspose2d(opt.latent_dim, 512, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, opt.channels, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        z = z.view(z.size(0), z.size(1), 1, 1)
        img = self.model(z)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(opt.channels, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=0, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, img):
        validity = self.model(img)
        return validity


def adversarial_loss(output, target):
    target = target.expand_as(output)
    loss = F.binary_cross_entropy(output, target)
    return loss

def save_model(opt, generator, discriminator, optimizer_G, optimizer_D, current_epoch):
    out = os.path.join(opt.model_path, "checkpoint_{}.tar".format(current_epoch))
    state = {
        'generator': generator.state_dict(),
        'discriminator': discriminator.state_dict(),
        'optimizer_G': optimizer_G.state_dict(),
        'optimizer_D': optimizer_D.state_dict(),
        'epoch': current_epoch
    }
    torch.save(state, out)

In [32]:
def train(opt):
    os.makedirs("images", exist_ok=True)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    cid = CustomImageDataset(opt.csv_file, opt.img_size)

    dataloader = DataLoader(
        cid,
        batch_size=opt.batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=opt.n_cpu,
    )

    # Define Generator and Discriminator
    generator = Generator().to(device)
    discriminator = Discriminator().to(device)

    # Checking if GPU is available to run code
    cuda = torch.cuda.is_available()
    if cuda:
        generator.cuda()
        discriminator.cuda()

    # Defining loss functions / optimizers
    optimizer_G = optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

    Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

    # Keeping track of both gennerator and discriminator losses
    generator_losses = []
    discriminator_losses = []

    counter = 0   # Will keep track of gen/disc training sequence
    g = 5   # Training epoch sequence for generator
    d = 1   # Training epoch sequence for discriminator

    for epoch in range(opt.n_epochs):
        for i, (imgs, _) in enumerate(dataloader):
            valid = Tensor(imgs.size(0), 1, 1, 1).fill_(1.0)
            fake = Tensor(imgs.size(0), 1, 1, 1).fill_(0.0)

            real_imgs = imgs.type(Tensor)

            optimizer_G.zero_grad()

            # Making noisy image from a gaussian distribution
            z = Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim)))

            # Generating fake images
            gen_imgs = generator(z)

            g_loss = adversarial_loss(discriminator(gen_imgs), valid)

            optimizer_D.zero_grad()

            real_loss = adversarial_loss(discriminator(real_imgs), valid)
            fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)

            d_loss = (real_loss + fake_loss) / 2

            # Training discriminator for 15 epochs, then train generator for 30 epochs 
            if counter < d or counter >= (g+d):
                if counter >= (g+d):   #reset counter
                    counter = 0
                d_loss.backward()
                optimizer_D.step()
                print("[Epoch %d/%d] [Batch %d/%d] [D IS TRAINING] [D loss: %.4f] [G loss: %.4f]"
                % (epoch+1, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item()))                

            elif counter < (g+d):
                g_loss.backward()
                optimizer_G.step()
                print("[Epoch %d/%d] [Batch %d/%d] [G IS TRAINING] [D loss: %.4f] [G loss: %.4f]"
                % (epoch+1, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item()))

            batches_done = epoch * len(dataloader) + i
            if batches_done % opt.sample_interval == 0:
                save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

            if (epoch + 1) % opt.save_interval == 0:
                save_model(opt, generator, discriminator, optimizer_G, optimizer_D, epoch + 1)

        generator_losses.append(g_loss.item())
        discriminator_losses.append(d_loss.item())
        counter+=1


    plt.plot(generator_losses, label='Generator Loss')
    plt.plot(discriminator_losses, label='Discriminator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    plt.savefig('plot.png')

In [33]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--n_epochs", type=int, default=200, help="Number of epochs of training")
    parser.add_argument("--batch_size", type=int, default=64, help="Size of the batches")
    parser.add_argument("--lr", type=float, default=0.0002, help="Learning rate for the optimizer")
    parser.add_argument("--b1", type=float, default=0.5, help="Beta1 parameter for the optimizer")
    parser.add_argument("--b2", type=float, default=0.999, help="Beta2 parameter for the optimizer")
    parser.add_argument("--n_cpu", type=int, default=8, help="Number of CPU threads for data loading")
    parser.add_argument("--latent_dim", type=int, default=100, help="Dimensionality of the latent space")
    parser.add_argument("--img_size", type=int, default=64, help="Size of each image dimension")
    parser.add_argument("--channels", type=int, default=1, help="Number of image channels")
    parser.add_argument("--sample_interval", type=int, default=10000, help="Interval between image samples")
    parser.add_argument("--csv_file", type=str, default="male.csv", help="CSV file containing image paths and labels")
    parser.add_argument("--save_interval", type=int, default=50, help="Interval between saving model")
    parser.add_argument('--model_path', type=str, default='/home/rnap/data/celeb/female/model', help='Saved model path') 
    
    opt = parser.parse_args(["--n_epochs", "500", "--batch_size", "64", "--lr", "0.0001", "--b1", "0.5",
                         "--b2", "0.999", "--n_cpu", "8", "--latent_dim", "100", "--img_size", "64", 
                         "--channels", "1", "--sample_interval", "5000", "--csv_file", "female_mac.csv",
                         "--save_interval", "50", "--model_path", "/Users/cristianespinosa/Documents/RESUME_Summer_2023/GAN_Project/RESUME_UCM_GAN/Model"])
    train(opt)

/Users/cristianespinosa/opt/anaconda3/envs/tensor/lib/python3.9/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[Epoch 1/500] [Batch 0/312] [D IS TRAINING] [D loss: 0.6781] [G loss: 0.7218]
[Epoch 1/500] [Batch 1/312] [D IS TRAINING] [D loss: 0.5342] [G loss: 0.7881]
[Epoch 1/500] [Batch 2/312] [D IS TRAINING] [D loss: 0.4287] [G loss: 0.9648]
[Epoch 1/500] [Batch 3/312] [D IS TRAINING] [D loss: 0.3643] [G loss: 1.0988]
[Epoch 1/500] [Batch 4/312] [D IS TRAINING] [D loss: 0.3380] [G loss: 1.1937]
[Epoch 1/500] [Batch 5/312] [D IS TRAINING] [D loss: 0.2916] [G loss: 1.2676]
[Epoch 1/500] [Batch 6/312] [D IS TRAINING] [D loss: 0.2688] [G loss: 1.4994]
[Epoch 1/500] [Batch 7/312] [D IS TRAINING] [D loss: 0.2400] [G loss: 1.6501]
[Epoch 1/500] [Batch 8/312] [D IS TRAINING] [D loss: 0.1965] [G loss: 1.6997]
[Epoch 1/500] [Batch 9/312] [D IS TRAINING] [D loss: 0.1855] [G loss: 1.7663]
[Epoch 1/500] [Batch 10/312] [D IS TRAINING] [D loss: 0.1660] [G loss: 1.8491]
[Epoch 1/500] [Batch 11/312] [D IS TRAINING] [D loss: 0.1482] [G loss: 1.9530]
[Epoch 1/500] [Batch 12/312] [D IS TRAINING] [D loss: 0.1260] 

[Epoch 1/500] [Batch 104/312] [D IS TRAINING] [D loss: 0.0052] [G loss: 5.2156]
[Epoch 1/500] [Batch 105/312] [D IS TRAINING] [D loss: 0.0052] [G loss: 5.2581]
[Epoch 1/500] [Batch 106/312] [D IS TRAINING] [D loss: 0.0044] [G loss: 5.4112]
[Epoch 1/500] [Batch 107/312] [D IS TRAINING] [D loss: 0.0045] [G loss: 5.4834]
[Epoch 1/500] [Batch 108/312] [D IS TRAINING] [D loss: 0.0047] [G loss: 5.5548]
[Epoch 1/500] [Batch 109/312] [D IS TRAINING] [D loss: 0.0040] [G loss: 5.5989]
[Epoch 1/500] [Batch 110/312] [D IS TRAINING] [D loss: 0.0040] [G loss: 5.5262]
[Epoch 1/500] [Batch 111/312] [D IS TRAINING] [D loss: 0.0042] [G loss: 5.4543]
[Epoch 1/500] [Batch 112/312] [D IS TRAINING] [D loss: 0.0040] [G loss: 5.5044]
[Epoch 1/500] [Batch 113/312] [D IS TRAINING] [D loss: 0.0039] [G loss: 5.5220]
[Epoch 1/500] [Batch 114/312] [D IS TRAINING] [D loss: 0.0040] [G loss: 5.5827]
[Epoch 1/500] [Batch 115/312] [D IS TRAINING] [D loss: 0.0043] [G loss: 5.5127]
[Epoch 1/500] [Batch 116/312] [D IS TRAI

[Epoch 1/500] [Batch 207/312] [D IS TRAINING] [D loss: 4.4536] [G loss: 0.0005]
[Epoch 1/500] [Batch 208/312] [D IS TRAINING] [D loss: 1.8084] [G loss: 0.0420]
[Epoch 1/500] [Batch 209/312] [D IS TRAINING] [D loss: 0.7527] [G loss: 3.0419]
[Epoch 1/500] [Batch 210/312] [D IS TRAINING] [D loss: 0.3847] [G loss: 2.5696]
[Epoch 1/500] [Batch 211/312] [D IS TRAINING] [D loss: 0.1897] [G loss: 2.5822]
[Epoch 1/500] [Batch 212/312] [D IS TRAINING] [D loss: 0.1260] [G loss: 2.3859]
[Epoch 1/500] [Batch 213/312] [D IS TRAINING] [D loss: 0.0907] [G loss: 2.4183]
[Epoch 1/500] [Batch 214/312] [D IS TRAINING] [D loss: 0.0662] [G loss: 2.7375]
[Epoch 1/500] [Batch 215/312] [D IS TRAINING] [D loss: 0.0470] [G loss: 3.1248]
[Epoch 1/500] [Batch 216/312] [D IS TRAINING] [D loss: 0.0430] [G loss: 3.4025]
[Epoch 1/500] [Batch 217/312] [D IS TRAINING] [D loss: 0.0330] [G loss: 3.5182]
[Epoch 1/500] [Batch 218/312] [D IS TRAINING] [D loss: 0.0348] [G loss: 3.5400]
[Epoch 1/500] [Batch 219/312] [D IS TRAI

[Epoch 1/500] [Batch 310/312] [D IS TRAINING] [D loss: 0.0047] [G loss: 5.6579]
[Epoch 1/500] [Batch 311/312] [D IS TRAINING] [D loss: 0.0037] [G loss: 5.7694]
[Epoch 2/500] [Batch 0/312] [D IS TRAINING] [D loss: 0.0045] [G loss: 5.5326]
[Epoch 2/500] [Batch 1/312] [D IS TRAINING] [D loss: 0.0042] [G loss: 5.4022]
[Epoch 2/500] [Batch 2/312] [D IS TRAINING] [D loss: 0.0048] [G loss: 5.5514]
[Epoch 2/500] [Batch 3/312] [D IS TRAINING] [D loss: 0.0042] [G loss: 5.8134]
[Epoch 2/500] [Batch 4/312] [D IS TRAINING] [D loss: 0.0046] [G loss: 5.7613]
[Epoch 2/500] [Batch 5/312] [D IS TRAINING] [D loss: 0.0049] [G loss: 5.6686]
[Epoch 2/500] [Batch 6/312] [D IS TRAINING] [D loss: 0.0042] [G loss: 5.6047]
[Epoch 2/500] [Batch 7/312] [D IS TRAINING] [D loss: 0.0040] [G loss: 5.7397]
[Epoch 2/500] [Batch 8/312] [D IS TRAINING] [D loss: 0.0038] [G loss: 5.8406]
[Epoch 2/500] [Batch 9/312] [D IS TRAINING] [D loss: 0.0039] [G loss: 5.9317]
[Epoch 2/500] [Batch 10/312] [D IS TRAINING] [D loss: 0.0041

...

[Epoch 10/500] [Batch 187/312] [D IS TRAINING] [D loss: 0.1773] [G loss: 1.8616]
[Epoch 10/500] [Batch 188/312] [D IS TRAINING] [D loss: 0.2346] [G loss: 1.9061]
[Epoch 10/500] [Batch 189/312] [D IS TRAINING] [D loss: 0.2716] [G loss: 1.9353]
[Epoch 10/500] [Batch 190/312] [D IS TRAINING] [D loss: 0.2461] [G loss: 2.6350]
[Epoch 10/500] [Batch 191/312] [D IS TRAINING] [D loss: 0.1878] [G loss: 1.7254]
[Epoch 10/500] [Batch 192/312] [D IS TRAINING] [D loss: 0.1844] [G loss: 2.1622]
[Epoch 10/500] [Batch 193/312] [D IS TRAINING] [D loss: 0.2155] [G loss: 2.8918]
[Epoch 10/500] [Batch 194/312] [D IS TRAINING] [D loss: 0.1680] [G loss: 1.8203]
[Epoch 10/500] [Batch 195/312] [D IS TRAINING] [D loss: 0.2137] [G loss: 1.8415]
[Epoch 10/500] [Batch 196/312] [D IS TRAINING] [D loss: 0.2417] [G loss: 3.0426]
[Epoch 10/500] [Batch 197/312] [D IS TRAINING] [D loss: 0.2859] [G loss: 1.2390]
[Epoch 10/500] [Batch 198/312] [D IS TRAINING] [D loss: 0.2299] [G loss: 3.4379]
[Epoch 10/500] [Batch 199/31

[Epoch 10/500] [Batch 289/312] [D IS TRAINING] [D loss: 0.3555] [G loss: 3.4979]
[Epoch 10/500] [Batch 290/312] [D IS TRAINING] [D loss: 0.3324] [G loss: 0.9153]
[Epoch 10/500] [Batch 291/312] [D IS TRAINING] [D loss: 0.2122] [G loss: 3.8745]
[Epoch 10/500] [Batch 292/312] [D IS TRAINING] [D loss: 0.1945] [G loss: 3.0470]
[Epoch 10/500] [Batch 293/312] [D IS TRAINING] [D loss: 0.5438] [G loss: 0.5689]
[Epoch 10/500] [Batch 294/312] [D IS TRAINING] [D loss: 0.6065] [G loss: 5.4159]
[Epoch 10/500] [Batch 295/312] [D IS TRAINING] [D loss: 1.2235] [G loss: 0.2281]
[Epoch 10/500] [Batch 296/312] [D IS TRAINING] [D loss: 1.5665] [G loss: 8.3964]
[Epoch 10/500] [Batch 297/312] [D IS TRAINING] [D loss: 0.5010] [G loss: 1.3510]
[Epoch 10/500] [Batch 298/312] [D IS TRAINING] [D loss: 0.3585] [G loss: 2.0115]
[Epoch 10/500] [Batch 299/312] [D IS TRAINING] [D loss: 0.3951] [G loss: 1.3602]
[Epoch 10/500] [Batch 300/312] [D IS TRAINING] [D loss: 0.3502] [G loss: 3.1564]
[Epoch 10/500] [Batch 301/31

[Epoch 11/500] [Batch 80/312] [D IS TRAINING] [D loss: 0.2722] [G loss: 4.3512]
[Epoch 11/500] [Batch 81/312] [D IS TRAINING] [D loss: 0.4121] [G loss: 0.9398]
[Epoch 11/500] [Batch 82/312] [D IS TRAINING] [D loss: 0.5271] [G loss: 4.7231]
[Epoch 11/500] [Batch 83/312] [D IS TRAINING] [D loss: 0.4776] [G loss: 0.7726]
[Epoch 11/500] [Batch 84/312] [D IS TRAINING] [D loss: 0.3038] [G loss: 4.7713]
[Epoch 11/500] [Batch 85/312] [D IS TRAINING] [D loss: 0.5511] [G loss: 0.7034]
[Epoch 11/500] [Batch 86/312] [D IS TRAINING] [D loss: 0.5796] [G loss: 5.0185]
[Epoch 11/500] [Batch 87/312] [D IS TRAINING] [D loss: 0.4533] [G loss: 0.9420]
[Epoch 11/500] [Batch 88/312] [D IS TRAINING] [D loss: 0.3941] [G loss: 3.3745]
[Epoch 11/500] [Batch 89/312] [D IS TRAINING] [D loss: 0.2357] [G loss: 1.9216]
[Epoch 11/500] [Batch 90/312] [D IS TRAINING] [D loss: 0.2264] [G loss: 3.8059]
[Epoch 11/500] [Batch 91/312] [D IS TRAINING] [D loss: 0.2714] [G loss: 1.2845]
[Epoch 11/500] [Batch 92/312] [D IS TRAI

[Epoch 11/500] [Batch 182/312] [D IS TRAINING] [D loss: 0.3206] [G loss: 3.2210]
[Epoch 11/500] [Batch 183/312] [D IS TRAINING] [D loss: 0.4711] [G loss: 0.8786]
[Epoch 11/500] [Batch 184/312] [D IS TRAINING] [D loss: 0.4578] [G loss: 5.2373]
[Epoch 11/500] [Batch 185/312] [D IS TRAINING] [D loss: 0.5464] [G loss: 0.7701]
[Epoch 11/500] [Batch 186/312] [D IS TRAINING] [D loss: 0.6238] [G loss: 5.9662]
[Epoch 11/500] [Batch 187/312] [D IS TRAINING] [D loss: 0.8306] [G loss: 0.4251]
[Epoch 11/500] [Batch 188/312] [D IS TRAINING] [D loss: 0.4458] [G loss: 4.0304]
[Epoch 11/500] [Batch 189/312] [D IS TRAINING] [D loss: 0.3638] [G loss: 1.5742]
[Epoch 11/500] [Batch 190/312] [D IS TRAINING] [D loss: 0.5306] [G loss: 0.9586]
[Epoch 11/500] [Batch 191/312] [D IS TRAINING] [D loss: 0.6139] [G loss: 3.7721]
[Epoch 11/500] [Batch 192/312] [D IS TRAINING] [D loss: 0.5522] [G loss: 0.9209]
[Epoch 11/500] [Batch 193/312] [D IS TRAINING] [D loss: 0.3769] [G loss: 3.8800]
[Epoch 11/500] [Batch 194/31

KeyboardInterrupt: 

In [ ]:
A = pd.read_csv('female_mac.csv')
A

In [ ]:
A.iloc[:,1]